# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0a7ce98b20>
├── 'a' --> tensor([[ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794]])
└── 'x' --> <FastTreeValue 0x7f0a7cf0e070>
    └── 'c' --> tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                        [-0.0965,  0.1015,  0.9169, -0.7006],
                        [ 0.2640, -1.0227,  0.0749, -2.6792]])

In [4]:
t.a

tensor([[ 0.0491, -0.6140,  1.0990],
        [-1.7220,  1.0001,  0.7794]])

In [5]:
%timeit t.a

76.1 ns ± 0.425 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0a7ce98b20>
├── 'a' --> tensor([[ 0.5530, -2.8247, -0.8587],
│                   [-1.2850,  1.9798,  0.1427]])
└── 'x' --> <FastTreeValue 0x7f0a7cf0e070>
    └── 'c' --> tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                        [-0.0965,  0.1015,  0.9169, -0.7006],
                        [ 0.2640, -1.0227,  0.0749, -2.6792]])

In [7]:
%timeit t.a = new_value

73.4 ns ± 0.747 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794]]),
    x: Batch(
           c: tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                      [-0.0965,  0.1015,  0.9169, -0.7006],
                      [ 0.2640, -1.0227,  0.0749, -2.6792]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0491, -0.6140,  1.0990],
        [-1.7220,  1.0001,  0.7794]])

In [11]:
%timeit b.a

63.4 ns ± 1.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1679,  0.0984, -0.4367],
               [-2.1443,  1.7546,  1.1358]]),
    x: Batch(
           c: tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                      [-0.0965,  0.1015,  0.9169, -0.7006],
                      [ 0.2640, -1.0227,  0.0749, -2.6792]]),
       ),
)

In [13]:
%timeit b.a = new_value

621 ns ± 10.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

993 ns ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 268 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

300 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

286 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0a7cf0e0a0>
├── 'a' --> tensor([[[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]],
│           
│                   [[ 0.0491, -0.6140,  1.0990],
│                    [-1.7220,  1.0001,  0.7794]]])
└── 'x' --> <FastTreeValue 0x7f0a7cf0efd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.2 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f09c68451f0>
├── 'a' --> tensor([[ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794],
│                   [ 0.0491, -0.6140,  1.0990],
│                   [-1.7220,  1.0001,  0.7794]])
└── 'x' --> <FastTreeValue 0x7f09c68452e0>
    └── 'c' --> tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                        [-0.0965,  0.1015,  0.9169, -0.7006],
                 

In [23]:
%timeit t_cat(trees)

43.9 µs ± 851 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.9 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]],
       
               [[ 0.0491, -0.6140,  1.0990],
                [-1.7220,  1.0001,  0.7794]]]),
    x: Batch(
           c: tensor([[[-0.5203, -0.2351,  0.0290,  1.6137],
                       [-0.0965,  0.1015,  0.9169, -0.7006],
                       [ 0.2640, -1.0227,  0.0749, -2.6792]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794],
               [ 0.0491, -0.6140,  1.0990],
               [-1.7220,  1.0001,  0.7794]]),
    x: Batch(
           c: tensor([[-0.5203, -0.2351,  0.0290,  1.6137],
                      [-0.0965,  0.1015,  0.9169, -0.7006],
                      [ 0.2640, -1.0227,  0.0749, -2.6792],
                      [-0.5203, -0.2351,  0.0290,  1.6137],
                      [-0.0965,  

In [28]:
%timeit Batch.cat(batches)

184 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

590 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
